In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.special as special
from scipy.integrate import quad

import os
import shutil
import pickle

MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rcdefaults()

#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rc('font',**{'family':'serif','serif':['Times']})
plt.rc('text', usetex=True)


plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


%matplotlib inline

In [2]:
from utils import *

import sympy as sp
sp.init_printing()


In [6]:
angles = sp.symbols('\\theta_S \phi_S \\iota \Phi')
angles_old = sp.symbols('\\theta_S \phi_S \\theta_L \phi_L')
ts = sp.symbols('t')

crossI = sp.lambdify([ts, *angles], Fcross(ts, *angles))
crossII = sp.lambdify([ts, *angles], Fcross(ts, *angles, sp.pi/4))
plusI = sp.lambdify([ts, *angles], Fplus(ts, *angles))
plusII = sp.lambdify([ts, *angles], Fplus(ts, *angles, sp.pi/4))

def cross_func(tt,thetaS,phiS,iota,Phi,arm='I'):
    if arm == 'I':
        return crossI(tt,thetaS,phiS,iota,Phi)
    if arm == 'II':
        return crossII(tt,thetaS,phiS,iota,Phi)
    
def plus_func(tt,thetaS,phiS,iota,Phi,arm='I'):
    if arm == 'I':
        return plusI(tt,thetaS,phiS,iota,Phi)
    if arm == 'II':
        return plusII(tt,thetaS,phiS,iota,Phi)

ThetaL_func = sp.lambdify([*angles], ThetaL(*angles))
incl_func = sp.lambdify([*angles_old], source_inclination(*angles_old))

PhiL_aux = sp.lambdify([*angles], PhiL(*angles))
def PhiL_func(thetaS,phiS,iota,Phi):
    return convert_angle(PhiL_aux(thetaS,phiS,iota,Phi))

pol_aux = sp.lambdify([*angles_old], source_polarization(*angles_old))
def pol_func(thetaS,phiS,thetaL,phiL):
    return convert_angle(pol_aux(thetaS,phiS,thetaL,phiL))

# some test values

thetaS = np.arccos(0.3)
phiS = 5.
thetaL = np.arccos(-0.2)
phiL = 4.

# conversion to inclination and polarization

incl_angle = incl_func(thetaS,phiS,thetaL,phiL)
pol_angle = pol_func(thetaS,phiS,thetaL,phiL)

val_angles = [
    thetaS, phiS,
    incl_angle,
    pol_angle
]

print(val_angles)

[1.2661036727794992, 5.0, 1.109619786440621, 5.113441543944784]


In [7]:
def pattern_coef(k, thetaS, phiS, iota, Phi, arm='I'):
    
    # computes Fourier coefs for the LISA pattern in the FD
    # if k>0: returns coefs for +k and -k
    
    if not isinstance(k, int):
        print('The first argument must be integer. It is the number of a Fourier coefficient')
        return

    aux_real = quad(
        lambda phi: plus_func(phi/2./np.pi, thetaS, phiS, iota, Phi, arm=arm) * np.cos(k*phi),
        0., 2*np.pi
    )[0]
    aux_imag = quad(
        lambda phi: plus_func(phi/2./np.pi, thetaS, phiS, iota, Phi, arm=arm) * np.sin(k*phi),
        0., 2*np.pi
    )[0]
    
    aux_real /= 2*np.pi
    aux_imag /= 2*np.pi
    
    plus_coef = (1 + np.cos(iota)**2)*(aux_real - 1j*aux_imag)
    
    
    aux_real = quad(
        lambda phi: cross_func(phi/2./np.pi, thetaS, phiS, iota, Phi, arm=arm) * np.cos(k*phi),
        0., 2*np.pi
    )[0]
    aux_imag = quad(
        lambda phi: cross_func(phi/2./np.pi, thetaS, phiS, iota, Phi, arm=arm) * np.sin(k*phi),
        0., 2*np.pi
    )[0]
    
    aux_real /= 2*np.pi
    aux_imag /= 2*np.pi
    
    cross_coef = -2*np.cos(iota)*(aux_real - 1j*aux_imag)
    
    if k == 0:
        return plus_coef + 1j*cross_coef 
    return plus_coef + 1j*cross_coef, plus_coef.conjugate() + 1j*cross_coef.conjugate()


In [10]:
from ipywidgets import interactive

tt = np.linspace(0, 1. ,100)

def f(thetaS=np.arccos(0.3), phiS=5., incl_angle=incl_angle, pol_phase=pol_angle):
    
    fig,axes = plt.subplots(ncols=2, nrows=1, figsize=(15,6))
    
    ax,ax1 = axes

    ax.plot(
        (1 + np.cos(incl_angle)**2)*plus_func(tt, thetaS, phiS, incl_angle, pol_phase),
        -2*np.cos(incl_angle)*cross_func(tt, thetaS, phiS, incl_angle, pol_phase),
    )

    ax.set_xlabel('$(1+\\cos^{2}{\\iota})F_+$')
    ax.set_ylabel('$-2\\cos{\\iota}F_\\times$')
    
    ax.set_aspect('equal')
    ax.set_xlim(-1.5,1.5)
    ax.set_ylim(-1.5,1.5)
    
    for ax in axes:

        ax.grid(True,linestyle=':',linewidth='1.')
        ax.xaxis.set_ticks_position('both')
        ax.yaxis.set_ticks_position('both')
        ax.tick_params('both',length=3,width=0.5,which='both',direction = 'in',pad=10)
        
    
    coef = pattern_coef(0, thetaS, phiS, incl_angle, pol_phase)
    ax1.scatter(np.real(coef), np.imag(coef), s=100)
    
    for k in coef_range[1:]:
        k = int(k)
        pos,neg = pattern_coef(k, thetaS, phiS, incl_angle, pol_phase)
        x,y = np.real(pos), np.imag(pos)
        p = ax1.scatter(x,y, s=100)
        ax1.annotate(str(k), xy=(x,y), color='black', 
            xytext=(5,5), textcoords="offset points")
        ax1.scatter(np.real(neg), np.imag(neg), s=100, marker='*', color=p.get_facecolors()[-1])

        
    ax1.set_xlabel('Re')
    ax1.set_ylabel('Im')
    ax1.set_xlim(-1.,1.)
    ax1.set_ylim(-1.,1.)
#     ax.plot(
#         tt, (1 + np.cos(incl_angle)**2)*plus_func(tt, thetaS, phiS, incl_angle, pol_phase),
#     )
#     ax1.set_xlabel('time')
#     ax1.set_ylabel('Fplus')
# #     ax1.set_xlim(-1.,1.)
#     ax1.set_ylim(-1.5,1.5)



interactive_plot = interactive(
    f, 
    thetaS=(0., np.pi), 
    phiS=(0., 2*np.pi), 
    incl_angle=(0., np.pi),
    pol_phase=(0.,2*np.pi)
)

output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

interactive(children=(FloatSlider(value=1.2661036727794992, description='thetaS', max=3.141592653589793), Floa…